<a href="https://colab.research.google.com/github/joel-rodgers/compiler-construction/blob/main/Miniproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GROUP MEMBERS (ICS4C)**
1.   122184-Joel Rodgers K'obara
2.   120226-Davies Nathan Akidiva
3.   115862-Richard Jeremy Githuba
4.   121660-Alex Wamai Kariuki
5.   120357-Bettyrose Mburu


**Our mini-project is two forms of parsing:**
*   Predictive Parsing - this  is a special form of recursive descent parsing, where no backtracking is required, so this can predict which products to use to replace the input string. 
*   Shift-reducing Parsing - is a process of reducing a string to the start symbol of a grammar






1.  **PREDICTIVE PARSING**




In [ ]:
// Input: input.txt

// S->Ab|Bc
// A->b
// B->a

#include <stdio.h>
#include <string.h>
#define TSIZE 128

int table[100][TSIZE];
char terminal[TSIZE];
char nonterminal[26];
struct product
{
    char str[100];
    int len;
} pro[20];

int no_pro;
char first[26][TSIZE];
char follow[26][TSIZE];
char first_rhs[100][TSIZE];

int isNT(char c)
{
    return c >= 'A' && c <= 'Z';
}

void readFromFile()
{
    FILE *fptr;
    fptr = fopen("text.txt", "r");
    char buffer[255];
    int i;
    int j;
    while (fgets(buffer, sizeof(buffer), fptr))
    {
        printf("%s", buffer);
        j = 0;
        nonterminal[buffer[0] - 'A'] = 1;
        for (i = 0; i < strlen(buffer) - 1; ++i)
        {
            if (buffer[i] == '|')
            {
                ++no_pro;
                pro[no_pro - 1].str[j] = '\0';
                pro[no_pro - 1].len = j;
                pro[no_pro].str[0] = pro[no_pro - 1].str[0];
                pro[no_pro].str[1] = pro[no_pro - 1].str[1];
                pro[no_pro].str[2] = pro[no_pro - 1].str[2];
                j = 3;
            }
            else
            {
                pro[no_pro].str[j] = buffer[i];
                ++j;
                if (!isNT(buffer[i]) && buffer[i] != '-' && buffer[i] != '>')
                {
                    terminal[buffer[i]] = 1;
                }
            }
        }
        pro[no_pro].len = j;
        ++no_pro;
    }
}
void add_FIRST_A_to_FOLLOW_B(char A, char B)
{
    int i;
    for (i = 0; i < TSIZE; ++i)
    {
        if (i != '^')
            follow[B - 'A'][i] = follow[B - 'A'][i] || first[A - 'A'][i];
    }
}
void add_FOLLOW_A_to_FOLLOW_B(char A, char B)
{
    int i;
    for (i = 0; i < TSIZE; ++i)
    {
        if (i != '^')
            follow[B - 'A'][i] = follow[B - 'A'][i] || follow[A - 'A'][i];
    }
}
void FOLLOW()
{
    int t = 0;
    int i, j, k, x;
    while (t++ < no_pro)
    {
        for (k = 0; k < 26; ++k)
        {
            if (!nonterminal[k])
                continue;
            char nt = k + 'A';
            for (i = 0; i < no_pro; ++i)
            {
                for (j = 3; j < pro[i].len; ++j)
                {
                    if (nt == pro[i].str[j])
                    {
                        for (x = j + 1; x < pro[i].len; ++x)
                        {
                            char sc = pro[i].str[x];
                            if (isNT(sc))
                            {
                                add_FIRST_A_to_FOLLOW_B(sc, nt);
                                if (first[sc - 'A']['^'])
                                    continue;
                            }
                            else
                            {
                                follow[nt - 'A'][sc] = 1;
                            }
                            break;
                        }
                        if (x == pro[i].len)
                            add_FOLLOW_A_to_FOLLOW_B(pro[i].str[0], nt);
                    }
                }
            }
        }
    }
}
void add_FIRST_A_to_FIRST_B(char A, char B)
{
    int i;
    for (i = 0; i < TSIZE; ++i)
    {
        if (i != '^')
        {
            first[B - 'A'][i] = first[A - 'A'][i] || first[B - 'A'][i];
        }
    }
}
void FIRST()
{
    int i, j;
    int t = 0;
    while (t < no_pro)
    {
        for (i = 0; i < no_pro; ++i)
        {
            for (j = 3; j < pro[i].len; ++j)
            {
                char sc = pro[i].str[j];
                if (isNT(sc))
                {
                    add_FIRST_A_to_FIRST_B(sc, pro[i].str[0]);
                    if (first[sc - 'A']['^'])
                        continue;
                }
                else
                {
                    first[pro[i].str[0] - 'A'][sc] = 1;
                }
                break;
            }
            if (j == pro[i].len)
                first[pro[i].str[0] - 'A']['^'] = 1;
        }
        ++t;
    }
}
void add_FIRST_A_to_FIRST_RHS__B(char A, int B)
{
    int i;
    for (i = 0; i < TSIZE; ++i)
    {
        if (i != '^')
            first_rhs[B][i] = first[A - 'A'][i] || first_rhs[B][i];
    }
}
// Calculates FIRST(ß) for each A->ß
void FIRST_RHS()
{
    int i, j;
    int t = 0;
    while (t < no_pro)
    {
        for (i = 0; i < no_pro; ++i)
        {
            for (j = 3; j < pro[i].len; ++j)
            {
                char sc = pro[i].str[j];
                if (isNT(sc))
                {
                    add_FIRST_A_to_FIRST_RHS__B(sc, i);
                    if (first[sc - 'A']['^'])
                        continue;
                }
                else
                {
                    first_rhs[i][sc] = 1;
                }
                break;
            }
            if (j == pro[i].len)
                first_rhs[i]['^'] = 1;
        }
        ++t;
    }
}
int main()
{
    readFromFile();
    follow[pro[0].str[0] - 'A']['$'] = 1;
    FIRST();
    FOLLOW();
    FIRST_RHS();
    int i, j, k;

    // display first of each variable
    printf("\n");
    for (i = 0; i < no_pro; ++i)
    {
        if (i == 0 || (pro[i - 1].str[0] != pro[i].str[0]))
        {
            char c = pro[i].str[0];
            printf("FIRST OF %c: ", c);
            for (j = 0; j < TSIZE; ++j)
            {
                if (first[c - 'A'][j])
                {
                    printf("%c ", j);
                }
            }
            printf("\n");
        }
    }

    // display follow of each variable
    printf("\n");
    for (i = 0; i < no_pro; ++i)
    {
        if (i == 0 || (pro[i - 1].str[0] != pro[i].str[0]))
        {
            char c = pro[i].str[0];
            printf("FOLLOW OF %c: ", c);
            for (j = 0; j < TSIZE; ++j)
            {
                if (follow[c - 'A'][j])
                {
                    printf("%c ", j);
                }
            }
            printf("\n");
        }
    }
    // display first of each variable ß
    // in form A->ß
    printf("\n");
    for (i = 0; i < no_pro; ++i)
    {
        printf("FIRST OF %s: ", pro[i].str);
        for (j = 0; j < TSIZE; ++j)
        {
            if (first_rhs[i][j])
            {
                printf("%c ", j);
            }
        }
        printf("\n");
    }
    terminal['$'] = 1;
    terminal['^'] = 0;

    printf("\n");
    printf("\n\t**************** LL(1) PARSING TABLE *******************\n");
    printf("\t--------------------------------------------------------\n");
    printf("%-10s", "");
    for (i = 0; i < TSIZE; ++i)
    {
        if (terminal[i])
            printf("%-10c", i);
    }
    printf("\n");
    int p = 0;
    for (i = 0; i < no_pro; ++i)
    {
        if (i != 0 && (pro[i].str[0] != pro[i - 1].str[0]))
            p = p + 1;
        for (j = 0; j < TSIZE; ++j)
        {
            if (first_rhs[i][j] && j != '^')
            {
                table[p][j] = i + 1;
            }
            else if (first_rhs[i]['^'])
            {
                for (k = 0; k < TSIZE; ++k)
                {
                    if (follow[pro[i].str[0] - 'A'][k])
                    {
                        table[p][k] = i + 1;
                    }
                }
            }
        }
    }
    k = 0;
    for (i = 0; i < no_pro; ++i)
    {
        if (i == 0 || (pro[i - 1].str[0] != pro[i].str[0]))
        {
            printf("%-10c", pro[i].str[0]);
            for (j = 0; j < TSIZE; ++j)
            {
                if (table[k][j])
                {
                    printf("%-10s", pro[table[k][j] - 1].str);
                }
                else if (terminal[j])
                {
                    printf("%-10s", "");
                }
            }
            ++k;
            printf("\n");
        }
    }
}


2. **SHIFT-REDUCING PARSING**



In [ ]:
#include <bits/stdc++.h>
using namespace std;

struct grammer
{
    char p[20];
    char prod[20];
} g[10];

int main()
{
    int i, stpos, j, k, l, m, o, p, f, r, np, tspos, cr;
    cout << "\nEnter Number of productions:";
    cin >> np;
    char sc, ts[10];
    cout << "\nEnter productions:\n";
    for (i = 0; i < np; i++)
    {
        cin >> ts;
        strncpy(g[i].p, ts, 1);
        strcpy(g[i].prod, &ts[3]);
    }
    char ip[10];
    cout << "\nEnter Input:";
    cin >> ip;
    int lip = strlen(ip);
    char stack[10];
    stpos = 0;
    i = 0;
    // moving input
    sc = ip[i];
    stack[stpos] = sc;
    i++;
    stpos++;

    cout << "\n\nStack\tInput\tAction";
    do
    {
        r = 1;
        while (r != 0)
        {
            cout << "\n";
            for (p = 0; p < stpos; p++)
            {
                cout << stack[p];
            }
            cout << "\t";
            for (p = i; p < lip; p++)
            {
                cout << ip[p];
            }

            if (r == 2)
                cout << "\tReduced";
            else
                cout << "\tShifted";
            r = 0;

            // try reducing
            for (k = 0; k < stpos; k++)
            {
                f = 0;
                for (l = 0; l < 10; l++)
                    ts[l] = '\0';
                tspos = 0;
                for (l = k; l < stpos; l++) // removing first caharcter
                {
                    ts[tspos] = stack[l];
                    tspos++;
                }

                // now compare each possibility with production
                for (m = 0; m < np; m++)
                {
                    cr = strcmp(ts, g[m].prod);

                    // if cr is zero then match is found
                    if (cr == 0)
                    {
                        for (l = k; l < 10; l++) // removing matched part from stack
                        {
                            stack[l] = '\0';
                            stpos--;
                        }

                        stpos = k;

                        // concatinate the string
                        strcat(stack, g[m].p);
                        stpos++;
                        r = 2;
                    }
                }
            }
        }
        // moving input
        sc = ip[i];
        stack[stpos] = sc;
        i++;
        stpos++;

    } while (strlen(stack) != 1 && stpos != lip);

    if (strlen(stack) == 1)
    {
        cout << "\n String Accepted";
    }
}